In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# Configuración de estilos
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("📊 ANÁLISIS Y LIMPIEZA DE DATOS DE VOZ")
print("="*50)

In [ ]:
# 1. CARGAR Y EXPLORAR DATOS INICIALES
df = pd.read_csv("resultados_voz.csv")
print("📂 DATOS CARGADOS:")
print(f"• Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
print(f"• Columnas: {list(df.columns)}")

# Mostrar primeras filas
print("\n🔍 PRIMERAS FILAS:")
display(df.head())

In [ ]:
# 2. ANÁLISIS EXPLORATORIO INICIAL
print("📈 ESTADÍSTICAS DESCRIPTIVAS INICIALES:")
display(df.describe())

print("\n🔍 INFORMACIÓN GENERAL DEL DATASET:")
print(df.info())

print("\n🎯 VALORES NULOS POR COLUMNA:")
null_summary = pd.DataFrame({
    'Valores_Nulos': df.isnull().sum(),
    'Porcentaje_Nulos': (df.isnull().sum() / len(df) * 100).round(2)
})
display(null_summary)

In [ ]:
# 3. VISUALIZACIÓN DE VALORES NULOS
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Heatmap de valores nulos
sns.heatmap(df.isnull(), yticklabels=False, cbar=True, cmap='viridis', ax=axes[0])
axes[0].set_title('Mapa de Valores Nulos (Amarillo = Nulo)')

# Gráfico de barras de valores nulos
null_summary['Valores_Nulos'].plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Cantidad de Valores Nulos por Columna')
axes[1].set_ylabel('Cantidad de Valores Nulos')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# 4. DISTRIBUCIÓN DE LAS VARIABLES PRINCIPALES
variables = ['F0_Hz', 'Jitter_porcentaje', 'Shimmer_porcentaje', 'HNR_dB']

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for i, var in enumerate(variables):
    # Histograma con KDE
    sns.histplot(df[var].dropna(), kde=True, ax=axes[i], color='skyblue', bins=15)
    axes[i].axvline(df[var].mean(), color='red', linestyle='--', label=f'Media: {df[var].mean():.2f}')
    axes[i].axvline(df[var].median(), color='green', linestyle='--', label=f'Mediana: {df[var].median():.2f}')
    axes[i].set_title(f'Distribución de {var}')
    axes[i].set_xlabel(var)
    axes[i].legend()

plt.tight_layout()
plt.show()

In [ ]:
# 5. DETECCIÓN Y ANÁLISIS DE OUTLIERS
print("🎯 DETECCIÓN DE OUTLIERS (Método IQR):")
print("="*40)

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

outliers_summary = {}

for i, var in enumerate(variables):
    # Calcular límites IQR
    Q1 = df[var].quantile(0.25)
    Q3 = df[var].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identificar outliers
    outliers = df[(df[var] < lower_bound) | (df[var] > upper_bound)]
    outliers_count = len(outliers)
    
    outliers_summary[var] = {
        'outliers': outliers_count,
        'porcentaje': (outliers_count / len(df) * 100),
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }
    
    # Boxplot
    sns.boxplot(y=df[var], ax=axes[i], color='lightgreen')
    axes[i].set_title(f'Boxplot de {var}\n(Outliers: {outliers_count})')
    axes[i].set_ylabel(var)

plt.tight_layout()
plt.show()

# Mostrar resumen de outliers
outliers_df = pd.DataFrame(outliers_summary).T
display(outliers_df)

In [ ]:
# 6. LIMPIEZA DE DATOS
print("🧹 PROCEDIMIENTO DE LIMPIEZA:")
print("="*40)

# Crear copia para limpieza
df_clean = df.copy()
filas_originales = len(df_clean)

# 6.1 Eliminar filas donde todos los valores numéricos son nulos
numeric_cols = ['F0_Hz', 'Jitter_porcentaje', 'Shimmer_porcentaje', 'HNR_dB']
df_clean = df_clean.dropna(subset=numeric_cols, how='all')

# 6.2 Manejar outliers - reemplazar con límites IQR
for var in numeric_cols:
    Q1 = df_clean[var].quantile(0.25)
    Q3 = df_clean[var].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Reemplazar outliers con los límites
    df_clean[var] = np.where(df_clean[var] < lower_bound, lower_bound, df_clean[var])
    df_clean[var] = np.where(df_clean[var] > upper_bound, upper_bound, df_clean[var])

# 6.3 Imputar valores nulos restantes con la mediana
for col in numeric_cols:
    if df_clean[col].isnull().sum() > 0:
        median_val = df_clean[col].median()
        df_clean[col].fillna(median_val, inplace=True)

filas_finales = len(df_clean)

print(f"• Filas originales: {filas_originales}")
print(f"• Filas después de limpieza: {filas_finales}")
print(f"• Filas eliminadas: {filas_originales - filas_finales}")
print(f"• Porcentaje conservado: {(filas_finales/filas_originales*100):.1f}%")

In [ ]:

# 7. ANÁLISIS POST-LIMPIEZA
print("📊 COMPARATIVO PRE/POST LIMPIEZA:")
print("="*40)

fig, axes = plt.subplots(2, 4, figsize=(20, 10))

for i, var in enumerate(variables):
    # Antes de limpieza
    axes[0, i].hist(df[var].dropna(), alpha=0.7, color='red', label='Antes', bins=15)
    axes[0, i].set_title(f'{var} - Antes')
    axes[0, i].legend()
    
    # Después de limpieza
    axes[1, i].hist(df_clean[var], alpha=0.7, color='green', label='Después', bins=15)
    axes[1, i].set_title(f'{var} - Después')
    axes[1, i].legend()

plt.tight_layout()
plt.show()

In [ ]:
# 8. ENRIQUECIMIENTO DE DATOS
print("🎨 ENRIQUECIMIENTO DE DATOS:")
print("="*40)

# 8.1 Clasificación de F0 por rangos vocales
def clasificar_voz(f0):
    if f0 < 120: return "Muy Grave"
    elif 120 <= f0 < 160: return "Grave" 
    elif 160 <= f0 < 200: return "Media-Grave"
    elif 200 <= f0 < 240: return "Media-Aguda"
    elif 240 <= f0 < 300: return "Aguda"
    else: return "Muy Aguda"

df_clean['Clasificacion_Voz'] = df_clean['F0_Hz'].apply(clasificar_voz)

# 8.2 Calidad vocal basada en HNR
def calidad_vocal(hnr):
    if hnr > 20: return "Excelente"
    elif 15 < hnr <= 20: return "Buena"
    elif 10 < hnr <= 15: return "Regular"
    else: return "Mala"

df_clean['Calidad_Vocal'] = df_clean['HNR_dB'].apply(calidad_vocal)

# 8.3 Índice de perturbación combinada
df_clean['Indice_Perturbacion'] = (
    df_clean['Jitter_porcentaje'] + df_clean['Shimmer_porcentaje']
)

# 8.4 Z-score normalizado
for col in numeric_cols:
    df_clean[f'{col}_ZScore'] = (df_clean[col] - df_clean[col].mean()) / df_clean[col].std()

print("✅ Columnas añadidas:")
new_cols = [col for col in df_clean.columns if col not in df.columns]
for col in new_cols:
    print(f"  • {col}")

display(df_clean.head())

In [ ]:
# 9. VISUALIZACIONES AVANZADAS
print("📈 VISUALIZACIONES AVANZADAS:")
print("="*40)

# 9.1 Matriz de correlación
plt.figure(figsize=(10, 8))
correlation_matrix = df_clean[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title('Matriz de Correlación entre Variables de Voz')
plt.tight_layout()
plt.show()

In [ ]:
# 9.2 Distribución por clasificación de voz
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Clasificación por F0
clasificacion_count = df_clean['Clasificacion_Voz'].value_counts()
axes[0,0].pie(clasificacion_count.values, labels=clasificacion_count.index, 
              autopct='%1.1f%%', startangle=90)
axes[0,0].set_title('Distribución por Clasificación de Voz (F0)')

# Calidad vocal
calidad_count = df_clean['Calidad_Vocal'].value_counts()
axes[0,1].bar(calidad_count.index, calidad_count.values, color=['green', 'lightgreen', 'orange', 'red'])
axes[0,1].set_title('Distribución por Calidad Vocal (HNR)')
axes[0,1].tick_params(axis='x', rotation=45)

# Relación Jitter vs Shimmer
scatter = axes[1,0].scatter(df_clean['Jitter_porcentaje'], df_clean['Shimmer_porcentaje'],
                           c=df_clean['HNR_dB'], cmap='viridis', alpha=0.6)
axes[1,0].set_xlabel('Jitter (%)')
axes[1,0].set_ylabel('Shimmer (%)')
axes[1,0].set_title('Jitter vs Shimmer (Color: HNR)')
plt.colorbar(scatter, ax=axes[1,0])

# Distribución de F0 por calidad vocal
sns.boxplot(data=df_clean, x='Calidad_Vocal', y='F0_Hz', ax=axes[1,1])
axes[1,1].set_title('Distribución de F0 por Calidad Vocal')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# 10. ANÁLISIS ESTADÍSTICO FINAL
print("📋 REPORTE FINAL DE CALIDAD DE DATOS:")
print("="*50)

print(f"📊 RESUMEN GENERAL:")
print(f"• Muestra final: {len(df_clean)} registros")
print(f"• Variables numéricas: {len(numeric_cols)}")
print(f"• Variables categóricas: {len(['Clasificacion_Voz', 'Calidad_Vocal'])}")
print(f"• Tasa de conservación: {(len(df_clean)/len(df)*100):.1f}%")

print(f"\n🎵 DISTRIBUCIÓN VOCAL:")
print(f"• F0 promedio: {df_clean['F0_Hz'].mean():.1f} Hz")
print(f"• Rango F0: {df_clean['F0_Hz'].min():.1f} - {df_clean['F0_Hz'].max():.1f} Hz")
print(f"• HNR promedio: {df_clean['HNR_dB'].mean():.1f} dB")

print(f"\n📈 CALIDAD VOCAL:")
calidad_summary = df_clean['Calidad_Vocal'].value_counts()
for calidad, count in calidad_summary.items():
    print(f"• {calidad}: {count} muestras ({(count/len(df_clean)*100):.1f}%)")

print(f"\n🔧 PERTURBACIONES:")
print(f"• Jitter promedio: {df_clean['Jitter_porcentaje'].mean():.3f}%")
print(f"• Shimmer promedio: {df_clean['Shimmer_porcentaje'].mean():.3f}%")
print(f"• Índice de perturbación promedio: {df_clean['Indice_Perturbacion'].mean():.3f}")

In [ ]:
# 11. GUARDAR DATOS LIMPIOS
output_file = "resultados_voz_limpio.csv"
df_clean.to_csv(output_file, index=False, encoding='utf-8')

print(f"\n💾 DATOS GUARDADOS:")
print(f"• Archivo: {output_file}")
print(f"• Filas: {len(df_clean)}")
print(f"• Columnas: {len(df_clean.columns)}")
print(f"• Tamaño: {(df_clean.memory_usage(deep=True).sum() / 1024 / 1024):.2f} MB")

print(f"\n🎯 COLUMNAS EN ARCHIVO FINAL:")
for i, col in enumerate(df_clean.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\n✅ ANÁLISIS COMPLETADO EXITOSAMENTE!")